Промаркируем топиками все единицы векторной базы

Инсталлы

In [1]:
! pip install --upgrade --quiet yandex-cloud-ml-sdk


Импорты

In [7]:
import os
from yandex_cloud_ml_sdk import YCloudML
import json
import pandas as pd
import ast


Функция для запросов к модели

In [13]:


folder_id = os.environ["FOLDER_ID"]
api_key = os.environ["API_KEY"]

sdk = YCloudML(folder_id=folder_id, auth=api_key)
model = sdk.models.completions("yandexgpt", model_version="rc")


valid_topics = 'проходной балл, меганаправления, количество бюджетных мест, цена года обучения, общежития, индивидуальные достижения, компанни-партнеры, стажировки, математика, программирование, кафедры, преподаватели, стипендии, корпуса, ремонт, айти-этаж, лаборатории, подача заявления, дни открытых дверей, чат приемки, чаты меганаправлений, языки программирования, баллы на общежитие, сертификаты на общежитие, бви, магистратура, военная кафедра, культурные мероприятия, спортивные секции, спортивные соревнования'.split(', ') + ['мусор']
def tag_query(query):
    with open('../research/prompts/tagging_prompt.md', 'r', encoding='utf-8') as f:
        system_prompt = f.read()
    return model.run(system_prompt + "\n\n Входной запрос:\n" + query).text
tag_query('''Маник сделала, крутой капец
''')

'мусор\n'

Слияние в один датафрейм

In [4]:
result = {'source': [], 'chunk_text': []}
result['source'] += list(pd.read_csv('../data/support/merged_df.csv')['root_global_id'])
result['chunk_text'] += list(pd.read_csv('../data/support/merged_df.csv')['dialog_plain'])
result['source'] += list(pd.read_csv('../data/support/pages_mai.csv')['url'])
result['chunk_text'] += list(pd.read_csv('../data/support/pages_mai.csv')['content'])
result['source'] += list(pd.read_csv('../data/support/pdf_df.csv')['doc_name'])
result['chunk_text'] += list(pd.read_csv('../data/support/pdf_df.csv')['dialog_plain'])
result['source'] += list(pd.read_csv('../data/support/vk_chunks_df.csv')['doc_name'])
result['chunk_text'] += list(pd.read_csv('../data/support/vk_chunks_df.csv')['dialog_plain'])
result_df = pd.DataFrame(result)


In [5]:
result_df

,source,chunk_text
0,chat2021_10,возможность заселение комната знакомых друг из...
1,chat2021_10,получение временной регистрация заселиться общ...
2,chat2021_10,подать документ указать результат экзамен прий...
3,chat2021_10,целое обучение классный радовать преподаватель...
4,chat2021_10,добавить ориентироваться проходной балл предыд...
...,...,...
3876,ФРЭЛА МАИ,[ФРЭЛА МАИ 💥 В рамках VI Фестиваля радиоэлектр...
3877,ФРЭЛА МАИ,"[ФРЭЛА МАИ Уже завтра в 13:00 Василий Егоров, ..."
3878,ФРЭЛА МАИ,[ФРЭЛА МАИ МАИ приглашает студентов и школьник...
3879,ФРЭЛА МАИ,[ФРЭЛА МАИ 🚀 Попробуй свои силы — участвуй в О...


Обработка диалогов

In [ ]:
def add_query(query):
    ans = tag_query(query)

    topics = ans.strip().replace('[', '').replace(']', '').replace('\n\n\u200b', '').split(', ')
    res = []
    for i in topics:
        if i  in valid_topics:
            res.append(i)
    topics = res

    print(f'{query} -> {topics}')
    return topics

result_df['topics'] = result_df['chunk_text'].apply(add_query)

query -> ['программирование', 'языки программирования', 'магистратура', 'кафедры', 'стажировки']
query -> ['общежития', 'подача заявления', 'индивидуальные достижения', 'магистратура']
query -> ['подача заявления', 'кафедры', 'математика', 'программирование', 'индивидуальные достижения', 'магистратура', 'военная кафедра', 'спортивные секции']
query -> ['общежития', 'количество бюджетных мест', 'подача заявления', 'проходной балл']
query -> ['проходной балл', 'общежития', 'бви', 'подача заявления', 'количество бюджетных мест']
query -> ['проходной балл', 'подача заявления', 'общежития']
query -> ['культурные мероприятия', 'спортивные секции']


KeyboardInterrupt: 

Обработка  постов